In [1]:
from typing import Dict, List, Optional, Tuple
from pathlib import Path
from os import path
from unicodedata import name
import warnings
import json
from cv2 import transform         #TODO: Uncomment this line
import numpy as np
import glob
import h5py
import pandas as pd
import collections
from torch.utils.data.dataset import Dataset
import os
import torch
from PIL import Image
from src.utils.utils import torch_center_and_normalize, sort_jointly, load_obj, load_text
# from torch._six import container_abcs, string_classes, int_classes

import trimesh
import math
from pytorch3d.structures import Meshes
from pytorch3d.renderer.mesh import Textures

from src.utils.utils import rotation_matrix

import torchvision.datasets as datasets
import random


/nfs4/vinit/miniconda3/envs/pytorch3d/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_dir ="/home/SharedData/Vinit/ZERO_SHOT_DATASET_SHREC13/train/img"

imageFolderDataset = datasets.ImageFolder(image_dir)

In [3]:
imageFolderDataset

Dataset ImageFolder
    Number of datapoints: 6320
    Root location: /home/SharedData/Vinit/ZERO_SHOT_DATASET_SHREC13/train/img

##### There are 6320 images that will be used for training

In [4]:
data_dir = "/home/SharedData/Vinit/ZERO_SHOT_DATASET_SHREC13/train"

data_dir = path.join(data_dir, "model")

synset_set = {
                synset
                for synset in os.listdir(data_dir)
                if path.isdir(path.join(data_dir, synset))
            }

In [5]:
synset_set

{'airplane',
 'ant',
 'barn',
 'bed',
 'beer-mug',
 'bench',
 'bicycle',
 'binoculars',
 'brain',
 'bridge',
 'bush',
 'butterfly',
 'cabinet',
 'car_sedan',
 'castle',
 'chair',
 'church',
 'computer_monitor',
 'couch',
 'dog',
 'dolphin',
 'door',
 'dragon',
 'duck',
 'face',
 'fish',
 'floor_lamp',
 'flower_with_stem',
 'guitar',
 'hand',
 'hat',
 'head',
 'helicopter',
 'horse',
 'hot_air_balloon',
 'hourglass',
 'house',
 'human-skeleton',
 'ice-cream-cone',
 'knife',
 'mailbox',
 'microscope',
 'motorbike',
 'palm_tree',
 'piano',
 'pickup_truck',
 'potted_plant',
 'rabbit',
 'race_car',
 'sailboat',
 'satellite',
 'screwdriver',
 'sea_turtle',
 'shark',
 'ship',
 'shoe',
 'shovel',
 'skateboard',
 'skull',
 'skyscraper',
 'snowman',
 'space_shuttle',
 'spider',
 'standing_bird',
 'submarine',
 'sword',
 'table',
 'tablelamp',
 'tent',
 'train',
 'tree',
 'truck',
 'tv',
 'umbrella',
 'vase',
 'wheel',
 'wine-bottle',
 'wineglass',
 'wrist-watch'}

In [7]:
classes = sorted(list(synset_set))
label_by_number = {k: v for v, k in enumerate(classes)}
label_by_number

{'airplane': 0,
 'ant': 1,
 'barn': 2,
 'bed': 3,
 'beer-mug': 4,
 'bench': 5,
 'bicycle': 6,
 'binoculars': 7,
 'brain': 8,
 'bridge': 9,
 'bush': 10,
 'butterfly': 11,
 'cabinet': 12,
 'car_sedan': 13,
 'castle': 14,
 'chair': 15,
 'church': 16,
 'computer_monitor': 17,
 'couch': 18,
 'dog': 19,
 'dolphin': 20,
 'door': 21,
 'dragon': 22,
 'duck': 23,
 'face': 24,
 'fish': 25,
 'floor_lamp': 26,
 'flower_with_stem': 27,
 'guitar': 28,
 'hand': 29,
 'hat': 30,
 'head': 31,
 'helicopter': 32,
 'horse': 33,
 'hot_air_balloon': 34,
 'hourglass': 35,
 'house': 36,
 'human-skeleton': 37,
 'ice-cream-cone': 38,
 'knife': 39,
 'mailbox': 40,
 'microscope': 41,
 'motorbike': 42,
 'palm_tree': 43,
 'piano': 44,
 'pickup_truck': 45,
 'potted_plant': 46,
 'rabbit': 47,
 'race_car': 48,
 'sailboat': 49,
 'satellite': 50,
 'screwdriver': 51,
 'sea_turtle': 52,
 'shark': 53,
 'ship': 54,
 'shoe': 55,
 'shovel': 56,
 'skateboard': 57,
 'skull': 58,
 'skyscraper': 59,
 'snowman': 60,
 'space_shuttle'

In [15]:
synset_ids = []
model_ids = []
synset_start_idxs = {}
synset_num_models = {}

for synset in synset_set:
    synset_start_idxs[synset] = len(synset_ids)
    for model in os.listdir(path.join(data_dir, synset)):
        synset_ids.append(synset)
        model_ids.append(model)
    model_count = len(synset_ids) - synset_start_idxs[synset]
    synset_num_models[synset] = model_count

In [18]:
model_ids

['m500.off',
 'm501.off',
 'm499.off',
 'm502.off',
 'm497.off',
 'm496.off',
 'm495.off',
 'm498.off',
 'm494.off',
 'm1459.off',
 'm1456.off',
 'm1454.off',
 'm1449.off',
 'm1453.off',
 'm1455.off',
 'm1448.off',
 'm1462.off',
 'm1461.off',
 'm1450.off',
 'm1451.off',
 'm1458.off',
 'm1452.off',
 'm1460.off',
 'm1457.off',
 'm592.off',
 'm597.off',
 'm595.off',
 'm593.off',
 'm596.off',
 'm594.off',
 'm831.off',
 'm834.off',
 'm833.off',
 'm840.off',
 'm841.off',
 'm829.off',
 'm835.off',
 'm832.off',
 'm836.off',
 'm839.off',
 'm830.off',
 'm838.off',
 'm837.off',
 'm842.off',
 'm843.off',
 'm1782.off',
 'm1780.off',
 'm1785.off',
 'm1786.off',
 'm1781.off',
 'm1787.off',
 'm1779.off',
 'm1784.off',
 'm1783.off',
 'm1788.off',
 'm621.off',
 'm620.off',
 'm624.off',
 'm623.off',
 'm619.off',
 'm617.off',
 'm618.off',
 'm622.off',
 'm1618.off',
 'm1617.off',
 'm1615.off',
 'm1616.off',
 'm1619.off',
 'm1184.off',
 'm1261.off',
 'm1273.off',
 'm1139.off',
 'm1239.off',
 'm1259.off',
 '

In [21]:
def _get_item_ids(idx) -> Dict:
        """
        Read a model by the given index.

        Args:
            idx: The idx of the model to be retrieved in the dataset.

        Returns:
            dictionary with following keys:
            - synset_id (str): synset id
            - model_id (str): model id
        """
        model = {}
        model["synset_id"] = synset_ids[idx]
        model["model_id"] = model_ids[idx]
        return model

In [22]:
_get_item_ids(0)

{'synset_id': 'wineglass', 'model_id': 'm500.off'}

In [24]:
## Try loading off files using pytorch3d
from pytorch3d.io import IO

mesh = IO().load_mesh("/home/SharedData/Vinit/ZERO_SHOT_DATASET_SHREC13/train/model/airplane/m1118.off")

In [25]:
verts, faces, aux = mesh

ValueError: not enough values to unpack (expected 3, got 1)

In [26]:
mesh

In [27]:
## Model from PIX3d
mesh2 = IO().load_mesh("/home/SharedData/Vinit/pix3d_preprocessed/train/model/bed/IKEA_BEDDINGE/model.obj")

In [28]:
mesh2

In [42]:
from pytorch3d.io import load_obj

verts, faces, aux = load_obj("/home/SharedData/Vinit/pix3d_preprocessed/train/model/bed/IKEA_BEDDINGE/model.obj")

In [53]:
verts

tensor([[-0.4184, -0.1006,  0.0562],
        [-0.3593, -0.1006,  0.0442],
        [-0.3593, -0.1006,  0.0562],
        ...,
        [ 0.3715, -0.1006, -0.1181],
        [ 0.3723, -0.1006,  0.1956],
        [ 0.3723, -0.1006, -0.1183]])

In [47]:
faces.verts_idx

tensor([[   0,    1,    2],
        [   1,    0,    3],
        [   0,    4,    3],
        ...,
        [2958, 2959, 2957],
        [2908, 2959, 2958],
        [2959, 2908, 2910]])

In [50]:
verts1 = mesh.verts_packed()
faces1 = mesh.faces_packed()

In [52]:
verts1

tensor([[0.0250, 0.0615, 0.2391],
        [0.0250, 0.0616, 0.2400],
        [0.0250, 0.0619, 0.2429],
        ...,
        [0.4466, 0.0250, 0.1881],
        [0.4466, 0.0610, 0.2353],
        [0.4442, 0.0610, 0.2353]])

In [31]:
mesh3

(tensor([[-0.4184, -0.1006,  0.0562],
         [-0.3593, -0.1006,  0.0442],
         [-0.3593, -0.1006,  0.0562],
         ...,
         [ 0.3715, -0.1006, -0.1181],
         [ 0.3723, -0.1006,  0.1956],
         [ 0.3723, -0.1006, -0.1183]]),
 Faces(verts_idx=tensor([[   0,    1,    2],
         [   1,    0,    3],
         [   0,    4,    3],
         ...,
         [2958, 2959, 2957],
         [2908, 2959, 2958],
         [2959, 2908, 2910]]), normals_idx=tensor([[-1, -1, -1],
         [-1, -1, -1],
         [-1, -1, -1],
         ...,
         [-1, -1, -1],
         [-1, -1, -1],
         [-1, -1, -1]]), textures_idx=tensor([[   0,    1,    2],
         [   1,    0,    3],
         [   4,    5,    6],
         ...,
         [8244, 8245, 8243],
         [8246, 8245, 8244],
         [8245, 8246, 8247]]), materials_idx=tensor([0, 0, 0,  ..., 0, 0, 0])),
 Properties(normals=None, verts_uvs=tensor([[-5.0965,  1.0390],
         [-0.0000, -0.0000],
         [-0.0000,  1.0390],
         ...

In [32]:
from pytorch3d.io import off_io

In [34]:
mesh4 = off_io("/home/SharedData/Vinit/ZERO_SHOT_DATASET_SHREC13/train/model/airplane/m1118.off")

TypeError: 'module' object is not callable

In [35]:
off_io?

Type:        module
String form: <module 'pytorch3d.io.off_io' from '/nfs4/vinit/miniconda3/envs/pytorch3d/lib/python3.8/site-packages/pytorch3d/io/off_io.py'>
File:        ~/miniconda3/envs/pytorch3d/lib/python3.8/site-packages/pytorch3d/io/off_io.py
Docstring:  
This module implements utility functions for loading and saving
meshes as .off files.

This format is introduced, for example, at
http://www.geomview.org/docs/html/OFF.html .
